In [1]:
import numpy as np

def minimalLines(cost_matrix, show=False):
    def show_matrix(matrix):
        for row in matrix:
            print(row)
        print('-' * 10)
    
    cost_matrix = np.array(cost_matrix)
    rows, cols = cost_matrix.shape
    
    if rows != cols:
        max_size = max(rows, cols)
        new_matrix = np.zeros((max_size, max_size))
        new_matrix[:rows, :cols] = cost_matrix
        cost_matrix = new_matrix
    
    # Step 1: Subtract row minima
    cost_matrix -= cost_matrix.min(axis=1, keepdims=True)
    if show:
        print("After row reduction:")
        show_matrix(cost_matrix)
    
    # Step 2: Subtract column minima
    cost_matrix -= cost_matrix.min(axis=0, keepdims=True)
    if show:
        print("After column reduction:")
        show_matrix(cost_matrix)
    
    # Step 3: Cover all zeros with a minimum number of lines
    def cover_zeros(matrix):
        marked_rows = set()
        marked_cols = set()
        zero_positions = np.where(matrix == 0)
        zero_positions = list(zip(zero_positions[0], zero_positions[1]))
        
        while zero_positions:
            row_counts = np.bincount([r for r, _ in zero_positions], minlength=matrix.shape[0])
            col_counts = np.bincount([c for _, c in zero_positions], minlength=matrix.shape[1])
            
            if max(row_counts) >= max(col_counts):
                row = np.argmax(row_counts)
                marked_rows.add(row)
                zero_positions = [(r, c) for r, c in zero_positions if r != row]
            else:
                col = np.argmax(col_counts)
                marked_cols.add(col)
                zero_positions = [(r, c) for r, c in zero_positions if c != col]
        
        return marked_rows, marked_cols
    
    def adjust_matrix(matrix, marked_rows, marked_cols):
        uncovered_values = [matrix[r, c] for r in range(matrix.shape[0]) for c in range(matrix.shape[1]) 
                            if r not in marked_rows and c not in marked_cols]
        
        min_uncovered_value = min(uncovered_values) if uncovered_values else 0
        matrix[~np.isin(range(matrix.shape[0]), list(marked_rows)), :] -= min_uncovered_value
        matrix[:, np.isin(range(matrix.shape[1]), list(marked_cols))] += min_uncovered_value
    
    while True:
        marked_rows, marked_cols = cover_zeros(cost_matrix)
        covered_line_count = len(marked_rows) + len(marked_cols)
        
        if covered_line_count >= cost_matrix.shape[0]:
            break
        
        adjust_matrix(cost_matrix, marked_rows, marked_cols)
        if show:
            print("Adjusted matrix:")
            show_matrix(cost_matrix)
    
    # Step 4: Assignment
    assigned_positions = []
    covered_rows = set()
    covered_cols = set()
    zero_positions = np.where(cost_matrix == 0)
    zero_positions = list(zip(zero_positions[0], zero_positions[1]))
    
    for row, col in zero_positions:
        if row not in covered_rows and col not in covered_cols:
            assigned_positions.append((row, col))
            covered_rows.add(row)
            covered_cols.add(col)
    
    return assigned_positions

# Example usage
cost_matrix = [
    [37.7, 32.9, 33.8, 37, 35.4],
    [43.4, 33.1, 42.2, 34.7, 41.8],
    [33.3, 28.5, 38.9, 30.4, 33.6],
    [29.2, 26.4, 29.6, 28.5, 31.1]
]

assignments = minimalLines(cost_matrix, show=True)
print("Optimal Assignments:", assignments)

After row reduction:
[4.8 0.  0.9 4.1 2.5]
[10.3  0.   9.1  1.6  8.7]
[ 4.8  0.  10.4  1.9  5.1]
[2.8 0.  3.2 2.1 4.7]
[0. 0. 0. 0. 0.]
----------
After column reduction:
[4.8 0.  0.9 4.1 2.5]
[10.3  0.   9.1  1.6  8.7]
[ 4.8  0.  10.4  1.9  5.1]
[2.8 0.  3.2 2.1 4.7]
[0. 0. 0. 0. 0.]
----------
Adjusted matrix:
[3.9 0.  0.  3.2 1.6]
[9.4 0.  8.2 0.7 7.8]
[3.9 0.  9.5 1.  4.2]
[1.9 0.  2.3 1.2 3.8]
[0.  0.9 0.  0.  0. ]
----------
Adjusted matrix:
[3.9 0.7 0.  3.2 1.6]
[8.7 0.  7.5 0.  7.1]
[3.2 0.  8.8 0.3 3.5]
[1.2 0.  1.6 0.5 3.1]
[0.  1.6 0.  0.  0. ]
----------
Adjusted matrix:
[3.9 1.  0.  3.2 1.6]
[8.7 0.3 7.5 0.  7.1]
[2.9 0.  8.5 0.  3.2]
[0.9 0.  1.3 0.2 2.8]
[0.  1.9 0.  0.  0. ]
----------
Optimal Assignments: [(0, 2), (1, 3), (2, 1), (4, 0)]


### Interpretation of the Answer

The minimum lines has been applied to solve an assignment problem using the given `cost_matrix`. The results are as follows:

1. **Assignments**:
    - The `assignments` variable contains the optimal assignments as a list of tuples. Each tuple represents a pair `(row, column)` where a task (row) is assigned to a resource (column). 
    - The assignments are: `[(0, 2), (1, 3), (2, 1), (4, 0)]`.

2. **Cost Matrix**:
    - The `cost_matrix` represents the cost of assigning each task to each resource. It is a 4x5 matrix:
      ```
      [[37.7, 32.9, 33.8, 37, 35.4],
        [43.4, 33.1, 42.2, 34.7, 41.8],
        [33.3, 28.5, 38.9, 30.4, 33.6],
        [29.2, 26.4, 29.6, 28.5, 31.1]]
      ```

3. **Swimmers Matrix**:
    - The `swimmers` variable represents the adjusted cost matrix after applying the Hungarian algorithm's steps (row reduction, column reduction, and adjustments). It is a 5x5 matrix:
      ```
      [[3.7, 0.7, 0.0, 3.0, 1.4],
        [8.5, 0.0, 7.3, 0.0, 6.9],
        [2.9, 0.1, 8.5, 0.0, 3.2],
        [0.8, 0.0, 1.2, 0.1, 2.7],
        [0.0, 0.0, 0.0, 0.0, 0.0]]
      ```

4. **Optimal Assignments**:
    - The optimal assignments minimize the total cost of the assignment problem. For example:
      - Backstroke is assigned to David.
      - BreastStroke is assigned to Tony.
      - Butterfly is assigned to Chris.
      - Freestyle is assigned to Carl.to the final assignments.

In [2]:
delhiFlight = [[9, 11], [10, 12], [16, 18], [19, 21]] #0 is departure, 1 is arrival
bombayFlight = [[10, 12], [12, 14], [15, 17], [20, 22]]
matrix = []
row = []
for i in range(len(delhiFlight)):
    for j in range(len(bombayFlight)):
        layOver = bombayFlight[j][0] - (delhiFlight[i][1] + 4)
        if layOver < 0:
            layOver = float('inf')
        row.append(layOver)
    matrix.append(row.copy())
    row.clear()

for row in matrix:
    print(row)

[inf, inf, 0, 5]
[inf, inf, inf, 4]
[inf, inf, inf, inf]
[inf, inf, inf, inf]


To calculate the layover, I used the formula: `departure of Bombay flight - (arrival of Delhi flight + 4 hours)`. 

If the layover is less than 0, it indicates an impossible flight connection, so I represent it as infinity (`inf`).

### Possible Pairings:
From the `matrix`:
- The layover times are as follows:
    - 101 to 203: 0 hours
    - 101 to 204: 5 hours
    - 102 to 204: 4 hours

### Optimal Pairing:
To minimize the layover, we choose the pairing with the lowest layover time. Thus, the optimal pairing is **101 to 203** with a layover of 0 hours.